In [1]:
import os

import pandas as pd

In [2]:
PATH = r'/Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum'

In [3]:
def get_df(file:str, columns: list = None, year_position:list = None):
    try:
        df = pd.read_csv(file, sep=';')

        if year_position:
            df['year'] = int(file[year_position[0]:year_position[1]])
            df['n_cult'] = len([col for col in list(df.columns) if '_int_' in col and col.startswith('y001')])
        
        if columns is not None:
            df = df[columns]

        return df
    except Exception as e:
        print(f"Error reading {file}: {e}")
        return None


def task(file:str):
    print(f"Reading file: {file}")
    return get_df(file = file, 
                  year_position = [-7, -4],
                  columns=['alpha_type',
                            'time',
                            'iter',
                            'alpha',
                            'status',
                            'OF_Optimized',
                            'net_income_total',
                            'ua_total',
                            'crop_diversity_total',
                            'year',
                            'n_cult'
                            ])


result = [task(os.path.join(PATH, file)) for file in os.listdir(PATH) if file.endswith('.csv')]

Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y008.csv
Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y009.csv
Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y010.csv
Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y004.csv
Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y005.csv
Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y011.csv
Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y007.csv
Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y013.csv
Reading file: /Users/robertopontes/Codes/CropRotation/results/data/al_rasheed/model_1/wsum/WeightedSum_y

In [4]:
df = pd.concat(result).sort_values(by=['year', 'alpha_type', 'iter'])

In [5]:
df_temp = df.copy()
df_temp['net_income_total'] = df_temp['net_income_total']/(10**6)
df_temp['const'] = (2*df_temp['n_cult'] + 21)*df_temp['year'] -2
df_temp['var'] = 2*df_temp['n_cult']*df_temp['year']
df_agg = df_temp.groupby(['year', 'const', 'var', 'alpha_type'])\
    .agg({  
            'time': ['mean', 'std'],
            'net_income_total': ['min', 'max', 'mean', 'std'],
            'ua_total': ['min', 'max', 'mean', 'std'],
            'crop_diversity_total': ['min', 'max', 'mean', 'std']
            }).round(2)
del df_temp

In [6]:
from itertools import product

for cols in list(product(('ua_total', 'crop_diversity_total'), ('mean',  'std'))):
    df_agg[cols] = df_agg[cols].apply(lambda x: int(round(x, 0)))

In [7]:
columns = {
    'time': 'Tempo de Execução (s)',
    'net_income_total': 'Retorno Líquido Total  (M \\textdollar)',
    'ua_total': 'U.A. Total',
    'crop_diversity_total': 'Quantidade de Culturas',
    'year': 'Qtd. Anos',
    'alpha_type': 'Variação de Alpha',
    'mean': 'Média',
    'std': 'DP',
    'min': 'Mínimo',
    'max': 'Máximo',
}

lines = {
    'alpha_lin' : 'Linear',
    'alpha_sig_004' : 'SIG004', # Sigmoidal ($\sigma = \\num{-0.004}$)
    'alpha_sig_06' : 'SIG06', # Sigmoidal ($\sigma = \\num{-0.06}$)
}

df_agg_final = df_agg.rename(columns=columns ).rename(lines)

In [8]:
for col in df_agg_final.select_dtypes(include='number').columns.tolist():
    df_agg_final[col] = df_agg_final[col].apply(lambda x: '\\num{'+ str(x) +'}')


In [9]:
df_agg_final.to_latex('summary_wsum.tex',
                longtable=True,
                caption='Summary',
                label='tab:summary',
                multirow=True)

In [10]:
df_agg_final

Tempo de Execução (s)              \
                                          Média          DP   
year const var alpha_type                                     
1    43    24  Linear                \num{0.02}   \num{0.0}   
               SIG004                \num{0.02}  \num{0.01}   
               SIG06                 \num{0.02}  \num{0.01}   
2    88    48  Linear                \num{0.03}  \num{0.01}   
               SIG004                \num{0.03}  \num{0.01}   
               SIG06                 \num{0.04}  \num{0.01}   
3    133   72  Linear                \num{0.06}  \num{0.06}   
               SIG004                \num{0.07}  \num{0.08}   
               SIG06                 \num{0.06}  \num{0.03}   
4    178   96  Linear                \num{0.13}  \num{0.19}   
               SIG004                 \num{0.2}  \num{0.26}   
               SIG06                 \num{0.12}  \num{0.09}   
5    223   120 Linear                \num{0.22}  \num{0.31}   
               SIG004                \num{0.29}  \num{0.41}   
               SIG06                 \num{0.14}  \num{0.13}   
6    268   144 Linear                \num{0.31}  \num{0.48}   
               SIG004                \num{0.39}  \num{0.57}   
               SIG06                 \num{0.16}  \num{0.17}   
7    313   168 Linear                \num{0.38}  \num{0.72}   
               SIG004                \num{0.52}  \num{0.88}   
               SIG06                 \num{0.17}  \num{0.24}   
8    358   192 Linear                \num{0.46}  \num{0.84}   
               SIG004                \num{0.67}  \num{1.14}   
               SIG06                  \num{0.2}  \num{0.34}   
9    403   216 Linear                \num{0.58}  \num{1.04}   
               SIG004                \num{0.78}  \num{2.55}   
               SIG06                  \num{0.2}  \num{0.37}   
10   448   240 Linear                \num{0.59}   \num{1.1}   
               SIG004                \num{0.91}  \num{1.49}   
               SIG06                 \num{0.25}   \num{0.5}   
11   493   264 Linear                \num{0.79}  \num{1.42}   
               SIG004                \num{1.03}  \num{1.64}   
               SIG06                 \num{0.29}  \num{0.54}   
12   538   288 Linear                \num{0.92}  \num{1.49}   
               SIG004                \num{1.14}  \num{1.56}   
               SIG06                 \num{0.31}  \num{0.55}   
13   583   312 Linear                \num{1.14}   \num{1.8}   
               SIG004                \num{1.44}  \num{1.72}   
               SIG06                  \num{0.4}  \num{0.61}   
14   628   336 Linear                 \num{1.9}  \num{7.94}   
               SIG004                 \num{1.6}   \num{2.2}   
               SIG06                  \num{0.5}  \num{3.91}   
15   673   360 Linear                \num{1.82}  \num{4.55}   
               SIG004                \num{2.13}  \num{3.19}   
               SIG06                 \num{0.58}  \num{3.14}   

                          Retorno Líquido Total  (M \textdollar)               \
                                                          Mínimo       Máximo   
year const var alpha_type                                                       
1    43    24  Linear                                 \num{3.65}   \num{6.33}   
               SIG004                                 \num{4.87}   \num{6.32}   
               SIG06                                  \num{3.65}   \num{6.33}   
2    88    48  Linear                                 \num{7.31}  \num{12.56}   
               SIG004                                  \num{9.5}  \num{12.54}   
               SIG06                                  \num{7.31}  \num{12.56}   
3    133   72  Linear                                \num{10.96}  \num{18.79}   
               SIG004                                \num{14.12}  \num{18.77}   
               SIG06                                 \num{10.96}  \num{18.79}   
4    178   96  Linear        